In [169]:
import nbimporter
import pandas as pd
import numpy as np
import warnings
import util_data_cleaner
import fetch_data_from_yfinance
import stock_list
warnings.filterwarnings('ignore')
from importlib import reload
reload(util_data_cleaner)
reload(fetch_data_from_yfinance)
reload(stock_list)
from datetime import date
import yfinance as yf

In [170]:
list_of_stocks = stock_list.get_list_of_stocks()
ipo_listing = util_data_cleaner.clean_ipo_symbol_data(list_of_stocks)
ipo_listing = ipo_listing.loc[ipo_listing['DATE OF LISTING']>='2020-01-01',]

In [176]:
b = []
for stock in ipo_listing['SYMBOL'].tolist():
    start_date = pd.to_datetime(ipo_listing.loc[ipo_listing['SYMBOL']==stock,'DATE OF LISTING'].values[0])
    end_date = date.today()
    interval = '1d'
    a = fetch_data_from_yfinance.data_puller(stock, start_date, end_date, interval)
    b.append(a)
final_data = pd.concat(b)
final_data.reset_index(inplace=True)


1 Failed download:
['BALCO.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['SUNLITE.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['POSITRON.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AESTHETIK.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['PPSL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['DHARIWAL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['UTSSAV.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['ASHALOG.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['BULKCORP.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['SSEGL.NS']: Exception('%ticker%: No timezone found, symbol may be deli

In [183]:
final_data.sort_values(by=['Stock Name', 'Date'], inplace=True)
final_data['Ipo_price'] = final_data.groupby('Stock Name')['Adj Close'].transform('first')
final_data['lower_bound'] = final_data['Ipo_price'] * 0.9
final_data['upper_bound'] = final_data['Ipo_price'] * 1.1
latest_close = final_data.groupby('Stock Name').tail(1)
latest_close['to_filter'] = 0
latest_close.loc[((latest_close['Adj Close'] >= latest_close['lower_bound']) & (latest_close['Adj Close'] <= latest_close['upper_bound'])),'to_filter'] = 1

final_data['away_from_ipo_price'] = final_data['Adj Close']/final_data['Ipo_price'] - 1
final_data['away_from_ipo_price_filter'] = final_data['away_from_ipo_price'].apply(lambda x: 1 if(x>0) else 0)
inter = final_data.groupby(['Stock Name'])['away_from_ipo_price_filter'].sum().reset_index()
latest_close = latest_close.merge(inter, how='left', on='Stock Name')

In [185]:
latest_close.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\strategy results\ipo_bo_23_aug_2024.csv', index=False)